# HashiCorp Vault Demo for Vault PKI Control Group

## Setup of the Demo
<img src="./images/control-group-overview.png" alt="Control Group Overview" width="500"/>

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

# Setting up HashiCorp Vault

In [1]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
# Expose the Vault API to the host machine.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=root" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-e "VAULT_LOG_LEVEL=DEBUG" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

8d676f535a4c8e096741c1332e5873673d5020ee2c30ddb9aef98a3a7e9778e8


# Vault PKI Secret Engine

<img src="./images/control-group-1.png" alt="Control Group 1" width="500"/>

## Step 1 - Enable Vault PKI Engine

In [2]:
# Login to Vault
vault login root

# Enable PKI secrets engine at the default mount path
vault secrets enable pki

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       QuZc3k811N0audH8clMe2ckq
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]
Success! Enabled the pki secrets engine at: pki/


## Step 2 - Setting up PKI Secret Engien

In [3]:
# Create the CA, the private key is kept inside Vault.
# Alternatively, generate an intermediate CA and sign with your root CA.
vault write pki/root/generate/internal \
    common_name=my-org.com \
    ttl=8760h

WARNING! The following warnings were returned from Vault:

  * This mount hasn't configured any authority information access (AIA)
  fields; this may make it harder for systems to find missing certificates
  in the chain or to validate revocation status of certificates. Consider
  updating /config/urls or the newly generated issuer with this information.

Key              Value
---              -----
certificate      -----BEGIN CERTIFICATE-----
MIIDMjCCAhqgAwIBAgIUdX5jQJ84gUAg2gaJOjDebeXZhHgwDQYJKoZIhvcNAQEL
BQAwFTETMBEGA1UEAxMKbXktb3JnLmNvbTAeFw0yNDA2MTgwMTM4MjNaFw0yNDA3
MjAwMTM4NTNaMBUxEzARBgNVBAMTCm15LW9yZy5jb20wggEiMA0GCSqGSIb3DQEB
AQUAA4IBDwAwggEKAoIBAQDgBBfEfsPh94PjSrKf2ARiF34BYfDWi4ZG0cQEShu8
YfZh5E/wMtLIbya9VJnNTU/dlu2YU4ivzE0fU16ykNj/qsDfyI8EilaXgt6ZH+Ym
vxJTxTkFi8xQX1zRhJYq9AfZ7KBFsgLwoclzhi0A6/lj/6P3VNjdVm4ZtgwRMB4g
cgRqdFh1b1C6mhAtTESkdCFpISHBWs6m7KZRZGAtdQhgo1wZq5VwP0N0a+Qkw0ev
SD3Uw+4xtiIQ3rfpwLLyQbyKMzRLzw8MhvPklcq7J0PLku9xKCSm6TiPaRr7CYqP
UKCV9NXSX7is/MMaaLpfU3KGjP/hKHJ

In [ ]:
# configure Vault with the URL's for CRL
vault write pki/config/urls \
    issuing_certificates="http://127.0.0.1:8200/v1/pki/ca" \
    crl_distribution_points="http://127.0.0.1:8200/v1/pki/crl"

In [ ]:
# configure wildcard certificate for *.website1.com
vault write pki/roles/website1role \
    allowed_domains=website1.com \
    allow_subdomains=true \
    max_ttl=72h

vault write pki/roles/website2role \
    allowed_domains=website2.com \
    allow_subdomains=true \
    max_ttl=72h

## Step 3 - Setting up Users - Approver & Requester

In [ ]:
## Create the Requester Policy with Control Group. Approver is the Security_Team Group.
vault policy write create_pki_cg -<<"EOF"
path "pki/issue/website1role" {
  capabilities = [ "create", "update" ]
 
  control_group = {
    factor "authorizer" {
      identity {
        group_names = [ "security_team" ]
        approvals = 1
      }
    }
  }
}

path "pki/issue/website2role" {
  capabilities = [ "create", "update" ]
}
EOF

In [ ]:
## Create the Approver Policy with Access to Control Group
vault policy write security_team -<<EOF
# To approve the request
path "sys/control-group/authorize" {
    capabilities = ["create", "update"]
}
 
# To check control group request status
path "sys/control-group/request" {
    capabilities = ["create", "update"]
}
EOF

In [5]:
## Enable Userpass for Demo Login
vault auth enable userpass

## Keep the Mount Accessor
vault auth list -format=json | jq -r '.["userpass/"].accessor'  > tmp/accessor.txt

Success! Enabled userpass auth method at: userpass/


In [ ]:
## Create User - Ron as part of the Security Team (Approver)

vault write auth/userpass/users/ron password="security"

vault write -format=json identity/entity name="Ron Teo" \
        policies="security_team" \
        metadata=team="Controller"

vault read -format=json identity/entity/name/"Ron Teo" | jq -r ".data.id" > tmp/entity_id_ron.txt

vault write identity/entity-alias name="ron" \
      canonical_id=$(cat tmp/entity_id_ron.txt) \
      mount_accessor=$(cat tmp/accessor.txt)

## Add Ron To Security_Team Group
vault write identity/group name="security_team" \
      policies="security_team" \
      member_entity_ids=$(cat tmp/entity_id_ron.txt)

In [ ]:
## Create User - Derek as part of the Developer Team (Requester)
vault write auth/userpass/users/derek password="developer"

vault write -format=json identity/entity name="Derek Tan" \
        policies="create_pki_cg" \
        metadata=team="Developer" 

vault read -format=json identity/entity/name/"Derek Tan" | jq -r ".data.id" > tmp/entity_id_derek.txt

vault write identity/entity-alias name="derek" \
      canonical_id=$(cat tmp/entity_id_derek.txt) \
      mount_accessor=$(cat tmp/accessor.txt)

## Step 4 - Requester Request PKI

<img src="./images/control-group-2.png" alt="Control Group 2" width="500"/>

In [ ]:
## Login Via Derek 
vault login -method=userpass username="derek" password="developer"

In [ ]:
# Triggered Control Group due to Policy
vault write pki/issue/website1role common_name=www.website1.com ttl=72 -format=json > tmp/wrapped_response.txt

## Extract and store the token & accessor
cat tmp/wrapped_response.txt
cat tmp/wrapped_response.txt | jq -r ".wrap_info.token" > tmp/wrapped_token.txt
cat tmp/wrapped_response.txt | jq -r ".wrap_info.accessor" > tmp/wrapped_accessor.txt

In [ ]:
## Certificate for Website2 can be created without approver.
vault write pki/issue/website2role common_name=www.website2.com ttl=72 -format=json

## Step 5 - Approver Approve PKI

<img src="./images/control-group-3.png" alt="Control Group 3" width="500"/>

In [ ]:
# Login Via Security Team
vault login -method=userpass username="ron" password="security"

In [ ]:
# Check the Request Information
vault write sys/control-group/request accessor=$(cat tmp/wrapped_accessor.txt)

In [ ]:
# Approve the Request
vault write sys/control-group/authorize accessor=$(cat tmp/wrapped_accessor.txt)

## Step 6 - Requester get PKI Certificate

<img src="./images/control-group-4.png" alt="Control Group 4" width="500"/>

In [ ]:
# Log back to Derek - Requester
vault login -method=userpass username="derek" password="developer"

In [ ]:
# Unwrapped the original Request Token
vault unwrap $(cat tmp/wrapped_token.txt)

## Option 1 - Vault Agent with PKI Control Group

In [ ]:
vault login root

vault auth enable approle
vault write auth/approle/role/cert-role token_policies="create_pki_cg" secret_id_ttl=24h token_ttl=5m token_max_ttl=4h
vault read -format=json auth/approle/role/cert-role/role-id > tmp/role.json
vault write -format=json -f auth/approle/role/cert-role/secret-id > tmp/secretid.json

cat tmp/role.json | jq -r .data.role_id > tmp/role.txt
cat tmp/secretid.json | jq -r .data.secret_id > tmp/secret.txt

export ROLE_ID="$(cat tmp/role.txt)" && echo $ROLE_ID 
export SECRET_ID="$(cat tmp/secret.txt)" && echo $SECRET_ID

In [ ]:
vault write auth/approle/login \
role_id=$ROLE_ID \
secret_id=$SECRET_ID

In [ ]:
tee tmp/agent-config.hcl <<EOF
pid_file = "./pidfile"

vault {
  address = "$VAULT_ADDR"
  retry {
    num_retries = 5
  }
}

auto_auth {
  method {
    type = "approle"
    config = {
      role_id_file_path = "tmp/role.txt"
      secret_id_file_path = "tmp/secret.txt"
      remove_secret_id_file_after_reading = false
    }
  }

  sink "file" {
    config = {
      path = "tmp/token"
    }
  }
}

cache {
  // An empty cache stanza still enables caching
}

template {
  contents = "{{ with pkiCert \"pki/issue/website1role\" \"common_name=www.website1.com\" \"ttl=1m\" }}{{ .Cert }}{{ end }}"
  destination = "tmp/cert.crt"
}

EOF

In [ ]:
vault agent -log-level debug -config=tmp/agent-config.hcl

# Cleanup

In [4]:
# Cleanup
vault login root

# Disable PKI secrets engine
vault secrets disable pki

# Disable Userpass
vault auth disable userpass

# Remove all the tmp file
rm -rf tmp/*

Success! You are now authenticated. The token information displayed below
is already stored in the token helper. You do NOT need to run "vault login"
again. Future Vault requests will automatically use this token.

Key                  Value
---                  -----
token                root
token_accessor       QuZc3k811N0audH8clMe2ckq
token_duration       ∞
token_renewable      false
token_policies       ["root"]
identity_policies    []
policies             ["root"]
Success! Disabled the secrets engine (if it existed) at: pki/
Success! Disabled the auth method (if it existed) at: userpass/


In [ ]:
# Stop Vault container
docker stop vault-enterprise